# Market Foundations & Financial Metrics - Chapter 01

This notebook accompanies [Part 1: Market Foundations](https://michaeltien8901.github.io/puffin/01-market-foundations/) of the Puffin tutorial.
It covers the essential financial metrics every algorithmic trader needs, using the `puffin` library.

**Topics covered:**
- Simple and log returns
- Annualized volatility
- Sharpe and Sortino ratios
- Drawdown analysis
- Alpha and beta
- Rolling metrics
- Multi-asset comparison
- Portfolio tearsheet via `puffin.portfolio`

## 1. Fetching Data with Puffin

We use `YFinanceProvider` from `puffin.data` rather than calling yfinance directly.
This ensures data flows through the same provider interface used by the rest of the system.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from puffin.data import YFinanceProvider

provider = YFinanceProvider()

# Fetch SPY and QQQ for comparison
symbols = ["SPY", "QQQ"]
data = provider.fetch_historical(symbols, start="2020-01-01", end="2024-01-01")
close = data["Close"].unstack("Symbol")
print(f"Loaded {len(close)} trading days for {list(close.columns)}")

close.plot(title="SPY vs QQQ", figsize=(12, 4), grid=True)
plt.ylabel("Price ($)")
plt.tight_layout()
plt.show()

## 2. Simple and Log Returns

**Simple returns** are used for portfolio calculations and reporting.
**Log returns** are additive across time, making them useful for statistical modeling.
For small daily moves, the two are nearly identical.

In [ ]:
# Simple returns
simple_returns = close.pct_change().dropna()

# Log returns
log_returns = np.log(close / close.shift(1)).dropna()

# Compare: they diverge for large moves
diff = (simple_returns - log_returns).abs()
print("Mean absolute difference between simple and log returns:")
print(diff.mean())
print(f"\nMax absolute difference: {diff.max().max():.6f}")

# Distribution of daily returns
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for i, sym in enumerate(symbols):
    simple_returns[sym].hist(bins=50, ax=axes[i], alpha=0.7)
    axes[i].set_title(f"{sym} Daily Returns")
    axes[i].set_xlabel("Return")
    axes[i].axvline(0, color="red", linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()

## 3. Annualized Volatility

Volatility measures dispersion of returns. We annualize by multiplying daily standard
deviation by sqrt(252) (trading days per year). Higher volatility means more risk.

In [ ]:
daily_vol = simple_returns.std()
annual_vol = daily_vol * np.sqrt(252)

print("Annualized Volatility:")
for sym in symbols:
    print(f"  {sym}: {annual_vol[sym]:.2%}")

# Rolling 60-day volatility
rolling_vol = simple_returns.rolling(60).std() * np.sqrt(252)
rolling_vol.plot(title="Rolling 60-Day Annualized Volatility", figsize=(12, 4), grid=True)
plt.ylabel("Volatility")
plt.tight_layout()
plt.show()

## 4. Sharpe and Sortino Ratios

The **Sharpe ratio** measures return per unit of total risk. The **Sortino ratio** only
penalizes downside volatility, which better reflects investor preferences.

| Sharpe | Interpretation |
|--------|---------------|
| < 0    | Losing money  |
| 0-1    | Below average |
| 1-2    | Good          |
| 2-3    | Very good     |
| > 3    | Suspect overfitting |

In [ ]:
risk_free_rate = 0.05  # 5% annual
rf_daily = risk_free_rate / 252

excess = simple_returns - rf_daily

# Sharpe ratio
sharpe = (excess.mean() / excess.std()) * np.sqrt(252)

# Sortino ratio (only penalizes downside)
downside = excess[excess < 0]
downside_std = excess.clip(upper=0).std()
sortino = (excess.mean() / downside_std) * np.sqrt(252)

# Calmar ratio (return / max drawdown)
ann_return = simple_returns.mean() * 252

print(f"{'Metric':<20} {'SPY':>10} {'QQQ':>10}")
print("-" * 42)
print(f"{'Ann. Return':<20} {ann_return['SPY']:>9.2%} {ann_return['QQQ']:>9.2%}")
print(f"{'Ann. Volatility':<20} {annual_vol['SPY']:>9.2%} {annual_vol['QQQ']:>9.2%}")
print(f"{'Sharpe Ratio':<20} {sharpe['SPY']:>10.2f} {sharpe['QQQ']:>10.2f}")
print(f"{'Sortino Ratio':<20} {sortino['SPY']:>10.2f} {sortino['QQQ']:>10.2f}")

## 5. Drawdown Analysis

Drawdown measures the decline from a peak. Maximum drawdown is the worst peak-to-trough
loss and tells you the most pain an investor would have experienced.

In [ ]:
equity = (1 + simple_returns).cumprod()
peak = equity.cummax()
drawdown = (equity - peak) / peak

for sym in symbols:
    max_dd = drawdown[sym].min()
    max_dd_date = drawdown[sym].idxmin()
    print(f"{sym}: Max Drawdown = {max_dd:.2%} on {max_dd_date.strftime('%Y-%m-%d')}")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
equity.plot(ax=ax1, title="Cumulative Returns (Growth of $1)")
ax1.grid(True, alpha=0.3)

drawdown.plot(ax=ax2, title="Drawdown")
ax2.fill_between(drawdown.index, drawdown["SPY"], 0, alpha=0.2)
ax2.fill_between(drawdown.index, drawdown["QQQ"], 0, alpha=0.2)
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Alpha and Beta

**Beta** measures sensitivity to the market (SPY). **Alpha** is the excess return after
accounting for market exposure. A positive alpha means outperformance.

In [ ]:
def alpha_beta(strategy, benchmark, rf_rate=0.05):
    """Compute annualized alpha and beta."""
    rf_daily = rf_rate / 252
    cov = np.cov(strategy.dropna(), benchmark.dropna())
    beta = cov[0, 1] / cov[1, 1]
    alpha = (
        (strategy.mean() - rf_daily) - beta * (benchmark.mean() - rf_daily)
    ) * 252
    return alpha, beta

alpha, beta = alpha_beta(simple_returns["QQQ"], simple_returns["SPY"])
print(f"QQQ vs SPY:")
print(f"  Beta:  {beta:.3f}  (>{1:.0f} = more volatile than market)")
print(f"  Alpha: {alpha:.4f}  ({alpha:.2%} annualized excess return)")

## 7. Rolling Sharpe Ratio

A rolling Sharpe ratio reveals how risk-adjusted performance changes over time.
This is more informative than a single-number summary.

In [ ]:
window = 126  # ~6 months
rolling_excess = simple_returns - rf_daily
rolling_sharpe = (
    rolling_excess.rolling(window).mean() / rolling_excess.rolling(window).std()
) * np.sqrt(252)

rolling_sharpe.plot(title=f"Rolling {window}-Day Sharpe Ratio", figsize=(12, 4), grid=True)
plt.axhline(0, color="red", linestyle="--", alpha=0.5)
plt.ylabel("Sharpe Ratio")
plt.tight_layout()
plt.show()

## 8. Portfolio Tearsheet with Puffin

The `puffin.portfolio` module provides `generate_tearsheet()` for comprehensive
performance analysis including Sharpe, Sortino, max drawdown, VaR, and more.

In [ ]:
from puffin.portfolio import generate_tearsheet, plot_returns

# Generate a full tearsheet for QQQ using SPY as benchmark
tearsheet = generate_tearsheet(
    simple_returns["QQQ"],
    benchmark=simple_returns["SPY"]
)

print("=== QQQ Performance Tearsheet ===")
for key, value in tearsheet.items():
    if isinstance(value, float):
        print(f"  {key:<30} {value:>10.4f}")
    else:
        print(f"  {key:<30} {value}")

# Plot cumulative returns
fig = plot_returns(simple_returns["QQQ"], benchmark=simple_returns["SPY"])
plt.show()

## 9. Multi-Asset Summary Table

Compare key metrics across multiple assets in a single DataFrame.

In [ ]:
# Fetch a broader universe
universe = ["SPY", "QQQ", "IWM", "GLD", "TLT"]
broad = provider.fetch_historical(universe, start="2020-01-01", end="2024-01-01")
broad_close = broad["Close"].unstack("Symbol")
broad_ret = broad_close.pct_change().dropna()

summary = pd.DataFrame(index=universe)
summary["Ann. Return"] = broad_ret.mean() * 252
summary["Ann. Vol"] = broad_ret.std() * np.sqrt(252)

excess_broad = broad_ret - rf_daily
summary["Sharpe"] = (excess_broad.mean() / excess_broad.std()) * np.sqrt(252)

eq = (1 + broad_ret).cumprod()
summary["Max DD"] = ((eq - eq.cummax()) / eq.cummax()).min()

# Beta vs SPY
for sym in universe:
    if sym == "SPY":
        summary.loc[sym, "Beta"] = 1.0
    else:
        cov = np.cov(broad_ret[sym].dropna(), broad_ret["SPY"].dropna())
        summary.loc[sym, "Beta"] = cov[0, 1] / cov[1, 1]

print(summary.round(3).to_string())

## Exercises

1. **Sector comparison**: Fetch XLK (tech), XLF (financials), XLE (energy) and compare Sharpe ratios. Which sector had the best risk-adjusted return?
2. **Drawdown duration**: Write a function that computes the longest drawdown period (in trading days). Which asset recovered slowest?
3. **Rolling beta**: Compute the 126-day rolling beta of QQQ vs SPY. Is the relationship stable over time?
4. **Win rate analysis**: Convert daily returns into trade P&L (positive days = wins). Compute win rate and profit factor for each asset.
5. **Crisis comparison**: Isolate the COVID crash (Feb-Mar 2020) and compute max drawdown and recovery time for each asset.